<a href="https://colab.research.google.com/github/JanMeow/GNN_StackedRF_3D_Spatial_DeepLearning/blob/main/Chpater_2_GNN_Guess_the_GeometricType_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy
!pip install pytorch
!pip install torch_geometric

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pytorch)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.6 MB/s eta 0:00:00


In [2]:
import torch
print(torch.__version__)

2.6.0+cu124


In [3]:
print(f"python {torch.version.cuda}")

python 12.4


In [4]:
torch.cuda.is_available()

False

In [5]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
Data(edge_index=[2, 4], x=[3, 1])

Data(x=[2], edge_index=[2])

#Transforming our graph into PYG graph

Although our data is in graph structure, it is different from the PyG format of graph we need for training.
So the first step would be to turn our graph into a PyG *graph*


1.   Turn node index into integers
2.   Use node.near property to connect the indexs



In [6]:
# Getting the data and data engineering functions
!git clone https://github.com/JanMeow/3d_graph_data_engineering.git
# Install the dependency
!mv 3d_graph_data_engineering data_engineering
!pip install numpy pandas ifcopenshell

Cloning into '3d_graph_data_engineering'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 104 (delta 61), reused 70 (delta 27), pack-reused 0 (from 0)
Receiving objects: 100% (104/104), 326.21 KiB | 5.35 MiB/s, done.
Resolving deltas: 100% (61/61), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.1 MB/s eta 0:00:00


In [7]:
import numpy as np
import pandas as pd
import ifcopenshell
import sys
sys.path.append('/content/data_engineering')
from data_engineering import collision as C, utils as U, compute_proxy as CP

In [8]:
from pathlib import Path
# lets use the test ifc_model
ifc_path = "/content/data_engineering/data/ifc/ML_train_data_0.ifc"

In [9]:
model = ifcopenshell.open(ifc_path)
root = model.by_type("IfcProject")[0]

In [10]:
from utils import  Graph
# Create a graph and establish BVH Tree=
graph = Graph.create(root)
graph.build_bvh()

Function ended, No more spatial child
Graph created


In [11]:
for node in graph.node_dict.values():
      if node.geom_info != None:
          node.near = [graph[guid] for guid in graph.bvh_query(node.geom_info["bbox"])
                        if guid != node.guid]

# Turn the data into features embedding for the GNN

In [12]:
#Get Intrisic features for all nodes
intrinsic_features = [CP.get_Intrinsic_features(graph, key) for key in graph.node_dict.keys()]
print("Finished Extracting Intrinsic Features")

Finished Extracting Intrinsic Features


In [13]:
#Example of an intrisic features
df1 = pd.DataFrame(intrinsic_features)
df1.head()

,AABB_X_Extent,OOBB_X_Extent,AABB_Y_Extent,OOBB_Y_Extent,AABB_Z_Extent,OOBB_Z_Extent,AABB_base_area,OOBB_Base_area,World_X_Extent,World_Y_Extent,...,Surface_area,Volume,Largest_face_area,Largest_face_normal_has_Z_vector,Number_of_vertices_in_base,Number_of_vertices,Number_of_faces,Number_of_XYZ_aligned_faces,Number_of_Z_aligned_faces,Number_of_faces_in_largest_face
0,4.69,4.651975,6.78,6.953317,0.04,0.040000,15.94,32.35,27.68,13.68,...,32.549824,0.659685,15.94,True,6,12,20,8,8,4
1,1.98,0.280000,3.70,0.202695,0.28,4.058783,0.80,0.06,27.68,13.68,...,3.953537,0.223608,1.12,False,4,8,12,4,4,2
2,3.44,4.458813,3.08,0.199449,0.28,0.280000,0.87,0.89,27.68,13.68,...,4.335963,0.244300,1.24,False,4,8,12,4,4,2
3,4.18,3.491121,6.53,6.070073,0.04,0.040000,15.59,21.19,27.68,13.68,...,31.833550,0.623736,15.59,True,5,10,16,6,6,3
4,0.57,0.280000,1.78,0.206350,0.28,1.831194,0.33,0.06,27.68,13.68,...,1.731608,0.092064,0.51,False,4,8,12,4,4,2


In [14]:
df1["Z_axis_aligned"] = df1["Z_axis_aligned"].apply(lambda x: 1 if x else 0)
df1["Largest_face_normal_has_Z_vector"] = df1["Largest_face_normal_has_Z_vector"].apply(lambda x: 1 if x else 0)
df1.head(2)

,AABB_X_Extent,OOBB_X_Extent,AABB_Y_Extent,OOBB_Y_Extent,AABB_Z_Extent,OOBB_Z_Extent,AABB_base_area,OOBB_Base_area,World_X_Extent,World_Y_Extent,...,Surface_area,Volume,Largest_face_area,Largest_face_normal_has_Z_vector,Number_of_vertices_in_base,Number_of_vertices,Number_of_faces,Number_of_XYZ_aligned_faces,Number_of_Z_aligned_faces,Number_of_faces_in_largest_face
0,4.69,4.651975,6.78,6.953317,0.04,0.040000,15.94,32.35,27.68,13.68,...,32.549824,0.659685,15.94,1,6,12,20,8,8,4
1,1.98,0.280000,3.70,0.202695,0.28,4.058783,0.80,0.06,27.68,13.68,...,3.953537,0.223608,1.12,0,4,8,12,4,4,2


In [15]:
x = []
for feature in intrinsic_features:
  x.append([v for v in feature.values()])
x = torch.tensor(x, dtype=torch.float)

In [16]:
x

tensor([[ 4.6900,  4.6520,  6.7800,  ...,  8.0000,  8.0000,  4.0000],
        [ 1.9800,  0.2800,  3.7000,  ...,  4.0000,  4.0000,  2.0000],
        [ 3.4400,  4.4588,  3.0800,  ...,  4.0000,  4.0000,  2.0000],
        ...,
        [ 0.0700,  0.0700,  5.1800,  ..., 42.0000, 10.0000,  8.0000],
        [ 0.2000,  0.2000,  0.2000,  ..., 12.0000,  4.0000,  2.0000],
        [ 0.2000,  0.2800,  3.6700,  ..., 10.0000,  4.0000,  2.0000]])

In [17]:
#Encode target labels
targets = [node.geom_type for node in graph.node_dict.values()]

In [18]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(targets)  # Now y is a NumPy array of integers
y = torch.tensor(y, dtype=torch.long)

In [19]:
# Create Different Edges
directions = ['upper', 'lower', 'left', 'right', 'near']
def encode_relation(relation):
    one_hot = [0] * len(directions)
    if relation in directions:
        one_hot[directions.index(relation)] = 1
    return one_hot

In [20]:
edge_index = []
edge_attr = []
for index, node in enumerate(graph.node_dict.values()):
  node.index = index
for node in graph.node_dict.values():
  for near_node in node.near:
    edge_index.append([node.index, near_node.index])
    edge_attr.append(encode_relation("near"))
  # Encode relation as one-hot (upper, lower, left, right)
  neighbours = CP.assign_neighbours(node, atol = 0.01, return_type = "guid")
  print(neighbours)
  for neighbour, direction in zip(neighbours, directions[0:4]):
    if neighbours != None:
      edge_index.append([node.index, neighbour])
      edge_attr.append(encode_relation(direction))
# edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

['3nybgsQSbEs8xcSjSdiYZd', '3nybgsQSbEs8xcSjSdiYZd', '3nybgsQSbEs8xcSjSdiYZd', '3nybgsQSbEs8xcSjSdiYZd']
['1nRaH42K52fBjm0EvslVLP', None, '1nRaH42K52fBjm0EvslVLP', '1nRaH42K52fBjm0EvslVLP']
['17Ca1esrb67BbaN7S1xyMh', '17Ca1esrb67BbaN7S1xyMh', '17Ca1esrb67BbaN7S1xyMh', '17Ca1esrb67BbaN7S1xyMh']
['3CcAbWEpj63wbgePaaSoY4', '3CcAbWEpj63wbgePaaSoY4', '3CcAbWEpj63wbgePaaSoY4', '3CcAbWEpj63wbgePaaSoY4']
['1K6v4JbBbDkBJkC1Sluu2u', None, '1K6v4JbBbDkBJkC1Sluu2u', '1K6v4JbBbDkBJkC1Sluu2u']
['2_7LMc0Xf7jgBWXXlkbTzE', '2_7LMc0Xf7jgBWXXlkbTzE', '2_7LMc0Xf7jgBWXXlkbTzE', '2_7LMc0Xf7jgBWXXlkbTzE']
['1FHoBBt_bFcA_4DQeUlxsx', '1FHoBBt_bFcA_4DQeUlxsx', None, '1FHoBBt_bFcA_4DQeUlxsx']
['1g5PqKudj7M86JP_MhAYUA', None, None, '1g5PqKudj7M86JP_MhAYUA']
['0F2Xa6lEn7mR7DECYVf7_x', '0F2Xa6lEn7mR7DECYVf7_x', '0F2Xa6lEn7mR7DECYVf7_x', '0F2Xa6lEn7mR7DECYVf7_x']
['2c_PCphkH0VwqZKKk1zN2b', '2c_PCphkH0VwqZKKk1zN2b', '2c_PCphkH0VwqZKKk1zN2b', '2c_PCphkH0VwqZKKk1zN2b']
['154wujlQ9B3QS$vDj2ApE1', None, None, '154wujlQ9B

In [21]:
data = Data(x=x, edge_index=edge_index, y=y)

In [22]:
data.num_node_features

34

# Split the data into k-fold for training and testing

In [23]:
from sklearn.model_selection import train_test_split

# Step 1: Split the node indices
num_nodes = data.num_nodes
node_indices = np.arange(num_nodes)

# Optional: Stratified sampling (preserves class balance)
train_idx, test_idx = train_test_split(
    node_indices,
    test_size=0.2,
    stratify=data.y.numpy(),  # ensures same label distribution
    random_state=42
)

In [24]:
# Create train/test masks
data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.train_mask[train_idx] = True
data.test_mask[test_idx] = True

# Create a GNN

In [25]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [26]:
model = GCN(input_dim=data.num_node_features, hidden_dim = 32, num_classes=len(le.classes_))

#Train the model

In [27]:
input_dim = data.num_node_features
hidden_dim = 32
num_classes = len(le.classes_)

model = GCN(input_dim, hidden_dim, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Training loop
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f}")


ValueError: `MessagePassing.propagate` only supports integer tensors of shape `[2, num_messages]`, `torch_sparse.SparseTensor` or `torch.sparse.Tensor` for argument `edge_index`.

# Evaluate the model

In [ ]:
model.eval()
with torch.no_grad():
    out = model(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = correct / data.test_mask.sum()
    print(f"\n✅ Test Accuracy: {acc:.4f}")

# Tuning the model

The model has only 60% accuracy at first but it shows that it is learning something.
In the Stacked RF two step classification apporach, even with just the intrisic features, the models could reach over 90% accuracy.
So first, lets see if we can up the performance by
1. Increasing the epochs
2. Increase the layers in the models

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Custom GCN block
class GCNBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = GCNConv(in_channels, out_channels)
        self.activation = nn.ReLU()

    def forward(self, x, edge_index):
        x = self.conv(x, edge_index)
        x = self.activation(x)
        return x

In [ ]:
class DeepGCN(nn.Module):
    def __init__(self, input_dim, hidden_dims, num_classes):
        super().__init__()

        self.layers = nn.ModuleList()
        dims = [input_dim] + hidden_dims

        for i in range(len(hidden_dims)):
            self.layers.append(GCNBlock(dims[i], dims[i + 1]))

        self.final_conv = GCNConv(dims[-1], num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        for layer in self.layers:
            x = layer(x, edge_index)
            x = F.dropout(x, p=0.5, training=self.training)

        x = self.final_conv(x, edge_index)
        return F.log_softmax(x, dim=1)

In [ ]:
input_dim = data.num_node_features
hidden_dim = [32, 64, 64] # this now takes a list
num_classes = len(le.classes_)

model = DeepGCN(input_dim, hidden_dim, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Training loop also increased the epoch
model.train()
for epoch in range(500):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f}")


In [ ]:
model.eval()
with torch.no_grad():
    out = model(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = correct / data.test_mask.sum()
    print(f"\n✅ Test Accuracy: {acc:.4f}")

# Graph Attention

since the model is not improving much by adding complexity,
it could be because now the averaging mechnicsm in Conv does not fit the case where certain nodes need more attention than the others.
In our case, we can try graph attention system

In [ ]:
from torch_geometric.nn import GATConv

class GATBlock(nn.Module):
    def __init__(self, in_channels, out_channels, heads =4, concat = True):
        super().__init__()
        self.gat = GATConv(in_channels, out_channels, heads= heads, concat= concat)
        self.activation = nn.ReLU()
        self.output_dim = out_channels * heads if concat else out_channels

    def forward(self, x, edge_index):
        x = self.gat(x, edge_index)
        x = self.activation(x)
        return x

In [ ]:
class DeepGAT(nn.Module):
    def __init__(self, input_dim, hidden_dims, num_classes, heads, concat):
        super().__init__()

        self.layers = nn.ModuleList()
        dims = [input_dim] + hidden_dims

        for i in range(len(hidden_dims)):
          concat = True if i < len(hidden_dims) - 1 else False  # Final layer? Don't concat
          block = GATBlock(dims[i], dims[i + 1], heads= heads, concat= concat)
          self.layers.append(block)
          dims[i + 1] = block.output_dim

        self.final_gat = GATConv(dims[-1], num_classes, heads=1, concat=False)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        for layer in self.layers:
            x = layer(x, edge_index)
            x = F.dropout(x, p=0.65, training=self.training)
        x = self.final_gat(x, edge_index)
        return F.log_softmax(x, dim=1)

In [ ]:
input_dim = data.num_node_features
hidden_dim = [32, 64, 64] # this now takes a list
num_classes = len(le.classes_)
heads = 4
concat = True

model = DeepGAT(input_dim, hidden_dim, num_classes, heads = heads, concat = concat)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Training loop also increased the epoch
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f}")


In [ ]:
model.eval()
with torch.no_grad():
    out = model(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = correct / data.test_mask.sum()
    print(f"\n✅ Test Accuracy: {acc:.4f}")

# Creating Edge Features